숙명여대 학과수집

In [5]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

In [6]:
# 수집할 컨텐츠가 있는 웹 페이지의 주소
url = "https://www.sookmyung.ac.kr/sookmyungkr/1030/subview.do"

In [7]:
# 접속객체 생성
session = requests.Session()
# 접속객체에 부가정보(header) 삽입하기
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})
# 생성한 접속객체를 활용하여 API에 접속
r = session.get(url)
# 접속에 실패한 경우
if r.status_code != 200:
# 에러코드와 에러메시지 출력
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
# 에러를 강제로 생성시킴
    raise Exception(msg)
r.encoding = "utf-8"
#print(r.text)
soup = BeautifulSoup(r.text)
# soup

응답결과로부터 데이터 추출하기

In [8]:
# 수집한 정보를 저장할 빈 리스트
학과목록_리스트 = []


college_list = soup.select(".college_list")


for item in college_list:
    h5El = item.select("h5")
    name = h5El[0].text.strip()

    viewEl = item.select(".view")
    view = viewEl[0].attrs["href"]
    if view.find("http://www.sookmyung.ac.kr") == -1:
        view = "http://www.sookmyung.ac.kr" + view


    # 학과소개 pdf
    pdfEl = item.select(".info") # soup.select하면 전체를 검토해서 info를가져오기 때문에
    pdf = pdfEl[0].attrs["href"] # item으로 한 학과 마다 검토를 해야 정확한 값이 나옴
    
    # 학과홈페이지
    homepageEl = item.select(".homepage")
    homepage = homepageEl[0].attrs['href']
    
    # 수집한 값들을 딕셔너리로 묶은 후 리스트에 추가
    college_dict = {"학과이름": name, "상세페이지": view, "홈페이지": homepage}

    #------------------------------------------------------# 상세보기 URL을 새롭게 수집
    #-------------------------------------------------------
    r = session.get(view)
    if r.status_code != 200:
    # 에러코드와 에러메시지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        print(msg)
        continue

    r.encoding = "utf-8"
    detailSoup = BeautifulSoup(r.text)
    #print(detailSoup)
    # 같은 구조를 갖는 전화번호, 팩스번호, 위치, 이메일주소를 가져온다.
    infoEl = detailSoup.select(".college_info_data dl")
    #print(infoEl)
    #print("-" * 50)
    # 딕셔너리 안에 for문을 돌리는 이유는 
    # 딕셔너리 값 안에 없는 것들을 찾아내서 딕셔너리에 추가하려고
    for info in infoEl:
    #print(info)
        dt = info.select("dt")[0].text.replace(":", "").strip() 
        dd = info.select("dd")[0].text.replace(":", "").strip()
        # info정보안에 dt항 아래에 0번째 택스트를 추출하는데
        # :항목에 앞 뒤 공간이 얼마나 있는 지 
        # 다른 항목들도 같은 공간을 차지하는 지 모르기때문에
        # 그냥 :삭제후 strip으로 같은 조건으로 만든다 
    #print(dt, dd)
    #print("-" * 50)
        college_dict[dt] = dd
    # 딕셔너리에 None값이면 값 추가 하니까 없는 항목에 대해서는 값추가
    #------------------------------------------------------# 
    # 상세보기 URL을 새롭게 수집 (끝)
    #-------------------------------------------------------
    학과목록_리스트.append(college_dict)
    # pdf파일 다운로드 -> URL을 저장하고 있는 변수는 pdf
    r = session.get(pdf, stream=True)
    if r.status_code == 200:
        r.encoding = "utf-8"
        with open("%s.pdf" % name , 'wb') as f:
            f.write(r.raw.read())


df = DataFrame(학과목록_리스트)
df.to_excel("result.xlsx")
df


,학과이름,상세페이지,홈페이지,전화,팩스,위치,이메일
0,화공생명공학부,http://www.sookmyung.ac.kr/sookmyungkr/1031/su...,http://cbe.sookmyung.ac.kr/,02-2077-7469,02-2077-7450,과학관463,chembioe@sookmyung.ac.kr
1,인공지능공학부,http://www.sookmyung.ac.kr/sookmyungkr/1033/su...,http://it.sookmyung.ac.kr/,02-710-9379,02-710-9704,명신관513,itdept@sookmyung.ac.kr
2,첨단소재·전자융합공학부지능형전자시스템전공,http://www.sookmyung.ac.kr/sookmyungkr/1034/su...,http://electro.sookmyung.ac.kr/,02-2077-7860,NaN,르네상스플라자 501,electro@sookmyung.ac.kr
3,첨단소재·전자융합공학부신소재물리전공,http://www.sookmyung.ac.kr/sookmyungkr/1035/su...,http://physics.sookmyung.ac.kr/,02-710-9407,02-2077-7320,과학관209,ca9407@sookmyung.ac.kr
4,소프트웨어학부컴퓨터과학전공,http://www.sookmyung.ac.kr/sookmyungkr/1037/su...,http://csweb.sookmyung.ac.kr/,02-710-9431,02-710-9296,명신관425,computer@sookmyung.ac.kr
5,소프트웨어학부데이터사이언스전공,http://www.sookmyung.ac.kr/sookmyungkr/1038/su...,http://ds.sookmyung.ac.kr/,02-710-9296,02-710-9296,명신관425,software@sookmyung.ac.kr
6,기계시스템학부,http://www.sookmyung.ac.kr/sookmyungkr/1039/su...,http://mse.sookmyung.ac.kr/,02-2077-7862,02-2077-7863,르네상스플라자 501,mechasys@sookmyung.ac.kr
7,기초공학부,http://www.sookmyung.ac.kr/sookmyungkr/1040/su...,http://basic.sookmyung.ac.kr/,02-2077-7855,NaN,르네상스플라자 501,basic@sookmyung.ac.kr
